In [22]:
import gzip

import mlflow
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split

import tensorflow
from tensorflow.keras.layers import Conv1D, Embedding, GlobalMaxPooling1D, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

import keras
from keras.layers import  Dropout

In [23]:
mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('intern-question/vector_lance')

<Experiment: artifact_location='/Users/lancesanterre/intern_2024/mlruns/1', creation_time=1724819171792, experiment_id='1', last_update_time=1724819171792, lifecycle_stage='active', name='intern-question/vector_lance', tags={}>

In [17]:
def weak_supervision_label(question):
    if not isinstance(question, str):
        #error handleing 
        return 

    question = question.lower()
    
    
    what_like = ["what", "which", "who", "where", "when"]
    how_like = ["how", "in what way", "by what means"]
    why_like = ["why", "for what reason", "how come"]
    
    
    label = np.array([0.0, 0.0, 0.0])
    
    # Define weights for the word position in the question
    weight_start = 1.5
    weight_middle = 1.0
    weight_end = 2.0
    
    # Helper function to apply weights based on position
    def apply_weights(word_list, weight):
        nonlocal label
        for word in word_list:
            if word in question:
                if question.endswith(word):
                    label += np.array([weight_end if word in what_like else 0, 
                                        weight_end if word in how_like else 0, 
                                        weight_end if word in why_like else 0])
                elif question.startswith(word):
                    label += np.array([weight_start if word in what_like else 0, 
                                        weight_start if word in how_like else 0, 
                                        weight_start if word in why_like else 0])
                else: 
                    label += np.array([weight_middle if word in what_like else 0, 
                                        weight_middle if word in how_like else 0, 
                                        weight_middle if word in why_like else 0])
    
    # Apply the weights for each category
    apply_weights(what_like, weight_middle)
    apply_weights(how_like, weight_middle)
    apply_weights(why_like, weight_middle)
    
    # Normalize the label to sum to 1
    if np.sum(label) > 0:
        label /= np.sum(label)
    else:
        label = np.array([0.33, 0.33, 0.33])  # Uncertain label
    
    return label.tolist()

In [18]:
with gzip.open('/Users/lancesanterre/intern_2024/data/cleaned/data.pkl', 'rb') as f:
    df = pd.read_pickle(f)
df

,0,labels
0,Which NFL team represented the AFC at Super Bo...,"[1.0, 0.0, 0.0]"
1,Which NFL team represented the NFC at Super Bo...,"[1.0, 0.0, 0.0]"
2,Where did Super Bowl 50 take place?,"[1.0, 0.0, 0.0]"
3,Which NFL team won Super Bowl 50?,"[1.0, 0.0, 0.0]"
4,What color was used to emphasize the 50th anni...,"[1.0, 0.0, 0.0]"
...,...,...
414916,How many keywords are there in the Racket prog...,"[0.0, 1.0, 0.0]"
414917,Do you believe there is life after death?,"[0.33, 0.33, 0.33]"
414918,What is one coin?,"[1.0, 0.0, 0.0]"
414919,What is the approx annual cost of living while...,"[1.0, 0.0, 0.0]"


In [19]:
def filter(df):
    # Convert the DataFrame to a NumPy array
    data_array = df.to_numpy()

    # Initialize lists to store rows based on the filter
    equal_to_33 = []
    not_equal_to_33 = []

    # Iterate over the rows in the array
    for row in data_array:
        # Check the condition in the 'labels' column (assumed to be the last column)
        labels = row[-1]  # Assuming 'labels' is the last column
        if all(label == 0.33 for label in labels):
            equal_to_33.append(row)
        else:
            not_equal_to_33.append(row)

    # Convert the lists back to DataFrames
    df_equal_to_33 = pd.DataFrame(equal_to_33, columns=df.columns)
    df_not_equal_to_33 = pd.DataFrame(not_equal_to_33, columns=df.columns)

    return df_equal_to_33, df_not_equal_to_33

In [20]:
unsure, sure = filter(df)
filtered_questions = sure[0]
filtered_labels = sure['labels']

# Simple LSTM

In [9]:
for input_dim in [1000, 2000, 4000, 5000]:
    for output_dim in [16, 32, 64, 128]:
        input_length = 10
        with mlflow.start_run():
            # Tokenization and Padding
            tokenizer = Tokenizer(num_words=input_dim)  # Adjust vocabulary size
            tokenizer.fit_on_texts(filtered_questions)
            sequences = tokenizer.texts_to_sequences(filtered_questions)
            X = pad_sequences(sequences, maxlen=input_length)

            # Convert labels to numpy array
            y = np.array(filtered_labels.tolist())

            # Model 1: Simple LSTM
            model_name = f"Simple_LSTM_{input_dim}_{output_dim}"
            model = Sequential([
                Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length),
                LSTM(output_dim),
                Dropout(0.5),
                Dense(32, activation='relu'),
                Dense(3, activation='softmax')
            ])

            # Compile and train
            model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

            # Evaluate and log
            loss, accuracy = model.evaluate(X_test, y_test)
            mlflow.log_params({"input_dim": input_dim, "output_dim": output_dim, "input_length": input_length})
            mlflow.log_metric("loss", loss)
            mlflow.log_metric("accuracy", accuracy)

            # Save the model with a unique name
            mlflow.keras.log_model(model, artifact_path=model_name)
            print(f"Model saved as {model_name} with accuracy: {accuracy:.2f}")
        mlflow.end_run()


Epoch 1/10


/opt/homebrew/anaconda3/envs/intern/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


8677/8677 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - accuracy: 0.9102 - loss: 0.2551 - val_accuracy: 0.9574 - val_loss: 0.1421
Epoch 2/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - accuracy: 0.9531 - loss: 0.1509 - val_accuracy: 0.9577 - val_loss: 0.1448
Epoch 3/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.9566 - loss: 0.1433 - val_accuracy: 0.9616 - val_loss: 0.1452
Epoch 4/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.9587 - loss: 0.1376 - val_accuracy: 0.9584 - val_loss: 0.1623
Epoch 5/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.9616 - loss: 0.1309 - val_accuracy: 0.9611 - val_loss: 0.1498
Epoch 6/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.9626 - loss: 0.1301 - val_accuracy: 0.9572 - val_loss: 0.1561
Epoch 7/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.9627 - loss: 0.1278 - val_accuracy: 0.9539 - val_loss: 0.1694
Epoch 8/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.9650 - loss: 0.1246 - val

2024/08/28 22:50:45 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
/opt/homebrew/anaconda3/envs/intern/lib/python3.11/site-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
2024/08/28 22:50:48 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Model saved as Simple_LSTM_1000_16 with accuracy: 0.96
Epoch 1/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.9222 - loss: 0.2240 - val_accuracy: 0.9588 - val_loss: 0.1386
Epoch 2/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.9571 - loss: 0.1418 - val_accuracy: 0.9614 - val_loss: 0.1326
Epoch 3/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.9610 - loss: 0.1323 - val_accuracy: 0.9625 - val_loss: 0.1306
Epoch 4/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.9631 - loss: 0.1268 - val_accuracy: 0.9642 - val_loss: 0.1276
Epoch 5/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.9663 - loss: 0.1213 - val_accuracy: 0.9651 - val_loss: 0.1269
Epoch 6/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.9677 - loss: 0.1173 - val_accuracy: 0.9663 - val_loss: 0.1268
Epoch 7/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.9691 - loss: 0.1137 - val_accuracy: 0.9661 - val_loss: 0.1264
Epoch 8/10
8677/8677 ━━━━━━━

...

Model saved as Simple_LSTM_5000_32 with accuracy: 0.95
Epoch 1/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 34s 4ms/step - accuracy: 0.9073 - loss: 0.2618 - val_accuracy: 0.9398 - val_loss: 0.1847
Epoch 2/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 33s 4ms/step - accuracy: 0.9484 - loss: 0.1651 - val_accuracy: 0.9450 - val_loss: 0.1691
Epoch 3/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 33s 4ms/step - accuracy: 0.9569 - loss: 0.1453 - val_accuracy: 0.9487 - val_loss: 0.1658
Epoch 4/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 33s 4ms/step - accuracy: 0.9628 - loss: 0.1311 - val_accuracy: 0.9502 - val_loss: 0.1741
Epoch 5/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 31s 4ms/step - accuracy: 0.9682 - loss: 0.1175 - val_accuracy: 0.9502 - val_loss: 0.1720
Epoch 6/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 32s 4ms/step - accuracy: 0.9720 - loss: 0.1078 - val_accuracy: 0.9506 - val_loss: 0.1814
Epoch 7/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 33s 4ms/step - accuracy: 0.9753 - loss: 0.1000 - val_accuracy: 0.9495 - val_loss: 0.2053
Epoch 8/10
8677/8677 ━━━━━━━

2024/08/29 00:02:17 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/08/29 00:02:19 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Model saved as Simple_LSTM_5000_64 with accuracy: 0.95
Epoch 1/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 77s 9ms/step - accuracy: 0.9167 - loss: 0.2397 - val_accuracy: 0.9443 - val_loss: 0.1743
Epoch 2/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 78s 9ms/step - accuracy: 0.9514 - loss: 0.1599 - val_accuracy: 0.9485 - val_loss: 0.1671
Epoch 3/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 79s 9ms/step - accuracy: 0.9595 - loss: 0.1391 - val_accuracy: 0.9501 - val_loss: 0.1680
Epoch 4/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 79s 9ms/step - accuracy: 0.9670 - loss: 0.1210 - val_accuracy: 0.9507 - val_loss: 0.1736
Epoch 5/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 79s 9ms/step - accuracy: 0.9721 - loss: 0.1073 - val_accuracy: 0.9508 - val_loss: 0.1741
Epoch 6/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 75s 9ms/step - accuracy: 0.9764 - loss: 0.0960 - val_accuracy: 0.9512 - val_loss: 0.1901
Epoch 7/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 80s 9ms/step - accuracy: 0.9797 - loss: 0.0873 - val_accuracy: 0.9506 - val_loss: 0.2099
Epoch 8/10
8677/8677 ━━━━━━━

2024/08/29 00:15:22 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/08/29 00:15:24 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Model saved as Simple_LSTM_5000_128 with accuracy: 0.95


#  Bi-directional LSTM

In [10]:
for input_dim in [1000, 2000, 4000, 5000]:
    for output_dim in [16, 32, 64, 128]:
        input_length = 10
        with mlflow.start_run():
            # Tokenization and Padding
            tokenizer = Tokenizer(num_words=input_dim)
            tokenizer.fit_on_texts(filtered_questions)
            sequences = tokenizer.texts_to_sequences(filtered_questions)
            X = pad_sequences(sequences, maxlen=input_length)
            y = np.array(filtered_labels.tolist())

            # Model 2: Bi-directional LSTM
            model_name = f"Bidirectional_LSTM_{input_dim}_{output_dim}"
            model = Sequential([
                Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length),
                Bidirectional(LSTM(output_dim)),
                Dropout(0.5),
                Dense(32, activation='relu'),
                Dense(3, activation='softmax')
            ])

            # Compile and train
            model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

            # Evaluate and log
            loss, accuracy = model.evaluate(X_test, y_test)
            mlflow.log_params({"input_dim": input_dim, "output_dim": output_dim, "input_length": input_length})
            mlflow.log_metric("loss", loss)
            mlflow.log_metric("accuracy", accuracy)

            # Save the model with a unique name
            mlflow.keras.log_model(model, artifact_path=model_name)
            print(f"Model saved as {model_name} with accuracy: {accuracy:.2f}")
        mlflow.end_run()


Epoch 1/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - accuracy: 0.9210 - loss: 0.2251 - val_accuracy: 0.9563 - val_loss: 0.1395
Epoch 2/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9563 - loss: 0.1416 - val_accuracy: 0.9610 - val_loss: 0.1325
Epoch 3/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9603 - loss: 0.1336 - val_accuracy: 0.9625 - val_loss: 0.1305
Epoch 4/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9622 - loss: 0.1297 - val_accuracy: 0.9627 - val_loss: 0.1309
Epoch 5/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9639 - loss: 0.1270 - val_accuracy: 0.9634 - val_loss: 0.1292
Epoch 6/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9653 - loss: 0.1230 - val_accuracy: 0.9629 - val_loss: 0.1289
Epoch 7/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9668 - loss: 0.1188 - val_accuracy: 0.9652 - val_loss: 0.1272
Epoch 8/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9672 - loss: 0

2024/08/29 00:18:31 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/08/29 00:18:34 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Model saved as Bidirectional_LSTM_1000_16 with accuracy: 0.96
Epoch 1/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 26s 3ms/step - accuracy: 0.9286 - loss: 0.2089 - val_accuracy: 0.9594 - val_loss: 0.1382
Epoch 2/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.9595 - loss: 0.1377 - val_accuracy: 0.9612 - val_loss: 0.1328
Epoch 3/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.9634 - loss: 0.1281 - val_accuracy: 0.9643 - val_loss: 0.1270
Epoch 4/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.9652 - loss: 0.1240 - val_accuracy: 0.9660 - val_loss: 0.1254
Epoch 5/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.9684 - loss: 0.1189 - val_accuracy: 0.9656 - val_loss: 0.1253
Epoch 6/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.9703 - loss: 0.1139 - val_accuracy: 0.9666 - val_loss: 0.1235
Epoch 7/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.9718 - loss: 0.1106 - val_accuracy: 0.9670 - val_loss: 0.1233
Epoch 8/10
8677/8677 

2024/08/29 00:22:50 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/08/29 00:22:52 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Model saved as Bidirectional_LSTM_1000_32 with accuracy: 0.97
Epoch 1/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 45s 5ms/step - accuracy: 0.9338 - loss: 0.1931 - val_accuracy: 0.9612 - val_loss: 0.1347
Epoch 2/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 44s 5ms/step - accuracy: 0.9623 - loss: 0.1327 - val_accuracy: 0.9636 - val_loss: 0.1284
Epoch 3/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 44s 5ms/step - accuracy: 0.9662 - loss: 0.1242 - val_accuracy: 0.9652 - val_loss: 0.1251
Epoch 4/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 45s 5ms/step - accuracy: 0.9689 - loss: 0.1177 - val_accuracy: 0.9652 - val_loss: 0.1271
Epoch 5/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 44s 5ms/step - accuracy: 0.9714 - loss: 0.1118 - val_accuracy: 0.9669 - val_loss: 0.1222
Epoch 6/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 44s 5ms/step - accuracy: 0.9732 - loss: 0.1065 - val_accuracy: 0.9681 - val_loss: 0.1228
Epoch 7/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 44s 5ms/step - accuracy: 0.9756 - loss: 0.1017 - val_accuracy: 0.9684 - val_loss: 0.1219
Epoch 8/10
8677/8677 

...

Model saved as Bidirectional_LSTM_5000_32 with accuracy: 0.95
Epoch 1/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 56s 6ms/step - accuracy: 0.9142 - loss: 0.2432 - val_accuracy: 0.9437 - val_loss: 0.1772
Epoch 2/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 54s 6ms/step - accuracy: 0.9496 - loss: 0.1655 - val_accuracy: 0.9451 - val_loss: 0.1692
Epoch 3/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 54s 6ms/step - accuracy: 0.9580 - loss: 0.1444 - val_accuracy: 0.9487 - val_loss: 0.1646
Epoch 4/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 50s 6ms/step - accuracy: 0.9638 - loss: 0.1295 - val_accuracy: 0.9497 - val_loss: 0.1698
Epoch 5/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 54s 6ms/step - accuracy: 0.9688 - loss: 0.1178 - val_accuracy: 0.9493 - val_loss: 0.1738
Epoch 6/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 54s 6ms/step - accuracy: 0.9729 - loss: 0.1069 - val_accuracy: 0.9495 - val_loss: 0.1815
Epoch 7/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 55s 6ms/step - accuracy: 0.9766 - loss: 0.0967 - val_accuracy: 0.9523 - val_loss: 0.1853
Epoch 8/10
8677/8677 

2024/08/29 02:28:52 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/08/29 02:28:54 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Model saved as Bidirectional_LSTM_5000_64 with accuracy: 0.95
Epoch 1/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 139s 16ms/step - accuracy: 0.9181 - loss: 0.2347 - val_accuracy: 0.9440 - val_loss: 0.1735
Epoch 2/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 138s 16ms/step - accuracy: 0.9514 - loss: 0.1581 - val_accuracy: 0.9493 - val_loss: 0.1630
Epoch 3/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 138s 16ms/step - accuracy: 0.9615 - loss: 0.1357 - val_accuracy: 0.9494 - val_loss: 0.1620
Epoch 4/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 137s 16ms/step - accuracy: 0.9683 - loss: 0.1181 - val_accuracy: 0.9527 - val_loss: 0.1674
Epoch 5/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 139s 16ms/step - accuracy: 0.9744 - loss: 0.1027 - val_accuracy: 0.9526 - val_loss: 0.1767
Epoch 6/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 134s 15ms/step - accuracy: 0.9785 - loss: 0.0913 - val_accuracy: 0.9525 - val_loss: 0.1905
Epoch 7/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 144s 17ms/step - accuracy: 0.9816 - loss: 0.0828 - val_accuracy: 0.9521 - val_loss: 0.2012
Epoch 8

2024/08/29 02:52:12 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/08/29 02:52:14 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Model saved as Bidirectional_LSTM_5000_128 with accuracy: 0.95


# CNN with Global Max pooling

In [24]:
for input_dim in [1000, 2000, 4000, 5000]:
    for output_dim in [16, 32, 64, 128]:
        input_length = 10
        with mlflow.start_run():
            # Tokenization and Padding
            tokenizer = Tokenizer(num_words=input_dim)
            tokenizer.fit_on_texts(filtered_questions)
            sequences = tokenizer.texts_to_sequences(filtered_questions)
            X = pad_sequences(sequences, maxlen=input_length)
            y = np.array(filtered_labels.tolist())

            # New Model: CNN with Global Max Pooling
            model_name = f"CNN_GlobalMaxPooling_{input_dim}_{output_dim}"
            model = Sequential([
                Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length),
                Conv1D(filters=output_dim, kernel_size=3, activation='relu'),
                GlobalMaxPooling1D(),
                Dropout(0.5),
                Dense(32, activation='relu'),
                Dense(3, activation='softmax')
            ])

            # Compile and train
            model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

            # Evaluate and log
            loss, accuracy = model.evaluate(X_test, y_test)
            mlflow.log_params({"input_dim": input_dim, "output_dim": output_dim, "input_length": input_length})
            mlflow.log_metric("loss", loss)
            mlflow.log_metric("accuracy", accuracy)

            # Save the model with a unique name
            mlflow.keras.log_model(model, artifact_path=model_name)
            print(f"Model saved as {model_name} with accuracy: {accuracy:.2f}")
        mlflow.end_run()


Epoch 1/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 4s 433us/step - accuracy: 0.8776 - loss: 0.3417 - val_accuracy: 0.9215 - val_loss: 0.3118
Epoch 2/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 4s 434us/step - accuracy: 0.9247 - loss: 0.2335 - val_accuracy: 0.9265 - val_loss: 0.3474
Epoch 3/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 4s 431us/step - accuracy: 0.9275 - loss: 0.2238 - val_accuracy: 0.9243 - val_loss: 0.2915
Epoch 4/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 4s 428us/step - accuracy: 0.9302 - loss: 0.2186 - val_accuracy: 0.9242 - val_loss: 0.3137
Epoch 5/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 4s 430us/step - accuracy: 0.9317 - loss: 0.2139 - val_accuracy: 0.9279 - val_loss: 0.2877
Epoch 6/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 4s 430us/step - accuracy: 0.9333 - loss: 0.2118 - val_accuracy: 0.9324 - val_loss: 0.2828
Epoch 7/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 4s 432us/step - accuracy: 0.9339 - loss: 0.2097 - val_accuracy: 0.9263 - val_loss: 0.2703
Epoch 8/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 4s 429us/step - accuracy: 0.9335 -

2024/08/29 10:21:37 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
/opt/homebrew/anaconda3/envs/intern/lib/python3.11/site-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
2024/08/29 10:21:39 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Model saved as CNN_GlobalMaxPooling_1000_16 with accuracy: 0.93
Epoch 1/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 7s 744us/step - accuracy: 0.9177 - loss: 0.2393 - val_accuracy: 0.9517 - val_loss: 0.1515
Epoch 2/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 7s 756us/step - accuracy: 0.9504 - loss: 0.1538 - val_accuracy: 0.9546 - val_loss: 0.1502
Epoch 3/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 6s 712us/step - accuracy: 0.9544 - loss: 0.1484 - val_accuracy: 0.9597 - val_loss: 0.1458
Epoch 4/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 6s 720us/step - accuracy: 0.9564 - loss: 0.1440 - val_accuracy: 0.9602 - val_loss: 0.1485
Epoch 5/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 6s 709us/step - accuracy: 0.9580 - loss: 0.1396 - val_accuracy: 0.9619 - val_loss: 0.1458
Epoch 6/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 6s 692us/step - accuracy: 0.9591 - loss: 0.1381 - val_accuracy: 0.9630 - val_loss: 0.1455
Epoch 7/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 6s 705us/step - accuracy: 0.9593 - loss: 0.1344 - val_accuracy: 0.9617 - val_loss: 0.1397
Epoch 8/10
8

2024/08/29 10:22:46 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/08/29 10:22:48 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Model saved as CNN_GlobalMaxPooling_1000_32 with accuracy: 0.96
Epoch 1/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.9303 - loss: 0.2125 - val_accuracy: 0.9608 - val_loss: 0.1399
Epoch 2/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - accuracy: 0.9579 - loss: 0.1434 - val_accuracy: 0.9625 - val_loss: 0.1358
Epoch 3/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - accuracy: 0.9622 - loss: 0.1355 - val_accuracy: 0.9644 - val_loss: 0.1329
Epoch 4/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.9633 - loss: 0.1320 - val_accuracy: 0.9658 - val_loss: 0.1331
Epoch 5/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.9658 - loss: 0.1267 - val_accuracy: 0.9657 - val_loss: 0.1313
Epoch 6/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.9670 - loss: 0.1248 - val_accuracy: 0.9662 - val_loss: 0.1296
Epoch 7/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.9674 - loss: 0.1242 - val_accuracy: 0.9661 - val_loss: 0.1319
Epoch 8/10
8677/8677 

...

Model saved as CNN_GlobalMaxPooling_5000_32 with accuracy: 0.95
Epoch 1/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.9058 - loss: 0.2673 - val_accuracy: 0.9421 - val_loss: 0.1809
Epoch 2/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9447 - loss: 0.1773 - val_accuracy: 0.9456 - val_loss: 0.1760
Epoch 3/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9504 - loss: 0.1634 - val_accuracy: 0.9446 - val_loss: 0.1746
Epoch 4/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.9562 - loss: 0.1513 - val_accuracy: 0.9460 - val_loss: 0.1755
Epoch 5/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9595 - loss: 0.1435 - val_accuracy: 0.9484 - val_loss: 0.1755
Epoch 6/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9624 - loss: 0.1363 - val_accuracy: 0.9501 - val_loss: 0.1772
Epoch 7/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9641 - loss: 0.1326 - val_accuracy: 0.9497 - val_loss: 0.1804
Epoch 8/10
8677/867

2024/08/29 10:51:18 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/08/29 10:51:20 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Model saved as CNN_GlobalMaxPooling_5000_64 with accuracy: 0.95
Epoch 1/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 27s 3ms/step - accuracy: 0.9138 - loss: 0.2478 - val_accuracy: 0.9453 - val_loss: 0.1758
Epoch 2/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.9473 - loss: 0.1688 - val_accuracy: 0.9488 - val_loss: 0.1685
Epoch 3/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 27s 3ms/step - accuracy: 0.9561 - loss: 0.1503 - val_accuracy: 0.9500 - val_loss: 0.1663
Epoch 4/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.9611 - loss: 0.1384 - val_accuracy: 0.9498 - val_loss: 0.1666
Epoch 5/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.9645 - loss: 0.1278 - val_accuracy: 0.9508 - val_loss: 0.1683
Epoch 6/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.9677 - loss: 0.1226 - val_accuracy: 0.9504 - val_loss: 0.1733
Epoch 7/10
8677/8677 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.9701 - loss: 0.1156 - val_accuracy: 0.9502 - val_loss: 0.1784
Epoch 8/10
8677/867

2024/08/29 10:55:41 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/08/29 10:55:43 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Model saved as CNN_GlobalMaxPooling_5000_128 with accuracy: 0.95
